# Neutrino Mixing Predictions in TSQVT

This notebook demonstrates the computation of the PMNS mixing matrix and neutrino oscillation parameters within the TSQVT framework.

**Reference:** Appendix O of "The Geometric Origin of Three Fermion Generations"

**Repository:** https://github.com/KerymMacryn/three-generations-repo

## Contents
1. Introduction and Setup
2. Experimental Data (NuFIT)
3. PMNS Matrix Construction
4. Dirac Neutrino Example
5. Type-I Seesaw Example
6. Comparison with Experiment
7. Uncertainty Propagation

## 1. Introduction and Setup

In [1]:
# Standard imports
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import eigh
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

# Import our module
import sys
sys.path.insert(0, '../scripts')
from pmns_mixing import (
    PMNSPredictor, construct_PMNS, extract_all_PMNS_parameters,
    build_neutrino_mass_matrix_dirac, build_neutrino_mass_matrix_seesaw,
    build_charged_lepton_mass_matrix, rephase_to_PDG,
    NUFIT_NO, NUFIT_IO, LEPTON_MASSES,
    monte_carlo_PMNS
)

print("Setup complete!")

Setup complete!


## 2. Experimental Data (NuFIT 5.2)

Current global fit results for neutrino oscillation parameters.

In [ ]:
# Display experimental values
print("NuFIT 5.2 Global Fit Results")
print("="*50)
print("\nNormal Ordering (NO):")
print(f"  θ₁₂ = {NUFIT_NO['theta12']:.2f}° ± {NUFIT_NO['theta12_err']:.2f}°")
print(f"  θ₂₃ = {NUFIT_NO['theta23']:.2f}° ± {NUFIT_NO['theta23_err']:.2f}°")
print(f"  θ₁₃ = {NUFIT_NO['theta13']:.2f}° ± {NUFIT_NO['theta13_err']:.2f}°")
print(f"  δ_CP = {NUFIT_NO['delta_CP']:.0f}° ± {NUFIT_NO['delta_CP_err']:.0f}°")
print(f"  Δm²₂₁ = {NUFIT_NO['Dm21_sq']:.2e} eV²")
print(f"  Δm²₃₁ = {NUFIT_NO['Dm31_sq']:.3e} eV²")

print("\nInverted Ordering (IO):")
print(f"  θ₁₂ = {NUFIT_IO['theta12']:.2f}° ± {NUFIT_IO['theta12_err']:.2f}°")
print(f"  θ₂₃ = {NUFIT_IO['theta23']:.2f}° ± {NUFIT_IO['theta23_err']:.2f}°")
print(f"  θ₁₃ = {NUFIT_IO['theta13']:.2f}° ± {NUFIT_IO['theta13_err']:.2f}°")
print(f"  δ_CP = {NUFIT_IO['delta_CP']:.0f}° ± {NUFIT_IO['delta_CP_err']:.0f}°")

In [ ]:
# Visualize the mixing angles
fig, ax = plt.subplots(figsize=(10, 5))

angles = ['θ₁₂', 'θ₂₃', 'θ₁₃']
no_values = [NUFIT_NO['theta12'], NUFIT_NO['theta23'], NUFIT_NO['theta13']]
no_errors = [NUFIT_NO['theta12_err'], NUFIT_NO['theta23_err'], NUFIT_NO['theta13_err']]
io_values = [NUFIT_IO['theta12'], NUFIT_IO['theta23'], NUFIT_IO['theta13']]
io_errors = [NUFIT_IO['theta12_err'], NUFIT_IO['theta23_err'], NUFIT_IO['theta13_err']]

x = np.arange(len(angles))
width = 0.35

ax.bar(x - width/2, no_values, width, yerr=no_errors, label='Normal Ordering',
       color='#3498db', capsize=5, alpha=0.8)
ax.bar(x + width/2, io_values, width, yerr=io_errors, label='Inverted Ordering',
       color='#e74c3c', capsize=5, alpha=0.8)

ax.set_ylabel('Angle (degrees)')
ax.set_title('Neutrino Mixing Angles from NuFIT 5.2')
ax.set_xticks(x)
ax.set_xticklabels(angles, fontsize=14)
ax.legend()

plt.tight_layout()
plt.show()

## 3. PMNS Matrix Construction

The PMNS matrix relates flavor and mass eigenstates:

$$U_{\mathrm{PMNS}} = U_\ell^\dagger U_\nu$$

In the PDG parameterization:

$$U_{\mathrm{PMNS}} = \begin{pmatrix}
c_{12}c_{13} & s_{12}c_{13} & s_{13}e^{-i\delta} \\
-s_{12}c_{23} - c_{12}s_{23}s_{13}e^{i\delta} & c_{12}c_{23} - s_{12}s_{23}s_{13}e^{i\delta} & s_{23}c_{13} \\
s_{12}s_{23} - c_{12}c_{23}s_{13}e^{i\delta} & -c_{12}s_{23} - s_{12}c_{23}s_{13}e^{i\delta} & c_{23}c_{13}
\end{pmatrix} \cdot P_{\mathrm{Maj}}$$

In [ ]:
def build_standard_PMNS(theta12, theta23, theta13, delta_CP):
    """
    Build PMNS matrix from mixing angles and CP phase.
    Angles in degrees.
    """
    # Convert to radians
    t12, t23, t13 = np.radians([theta12, theta23, theta13])
    d = np.radians(delta_CP)
    
    c12, s12 = np.cos(t12), np.sin(t12)
    c23, s23 = np.cos(t23), np.sin(t23)
    c13, s13 = np.cos(t13), np.sin(t13)
    
    U = np.array([
        [c12*c13, s12*c13, s13*np.exp(-1j*d)],
        [-s12*c23 - c12*s23*s13*np.exp(1j*d), 
         c12*c23 - s12*s23*s13*np.exp(1j*d), 
         s23*c13],
        [s12*s23 - c12*c23*s13*np.exp(1j*d),
         -c12*s23 - s12*c23*s13*np.exp(1j*d),
         c23*c13]
    ])
    
    return U

# Build experimental PMNS matrix
U_exp = build_standard_PMNS(
    NUFIT_NO['theta12'], 
    NUFIT_NO['theta23'],
    NUFIT_NO['theta13'],
    NUFIT_NO['delta_CP']
)

print("Experimental PMNS matrix (Normal Ordering):")
print("")
for i in range(3):
    row = [f"{U_exp[i,j].real:+.4f}{U_exp[i,j].imag:+.4f}j" for j in range(3)]
    print(f"  [{', '.join(row)}]")

## 4. Dirac Neutrino Example

In the Dirac case, neutrino masses are:

$$M_\nu = Y_\nu \cdot \mathrm{diag}(\rho_1, \rho_2, \rho_3)$$

In [ ]:
# Vacuum condensation values from quark sector fit
rho = np.array([0.0003, 0.06, 0.98])

print("Vacuum condensation values (from TSQVT):")
print(f"  ρ₁ = {rho[0]:.4f}")
print(f"  ρ₂ = {rho[1]:.4f}")
print(f"  ρ₃ = {rho[2]:.4f}")

In [ ]:
# Example: Tribimaximal-like structure
# This gives θ₁₂ ≈ 35.3°, θ₂₃ = 45°, θ₁₃ = 0

s12_tb = 1/np.sqrt(3)
c12_tb = np.sqrt(2/3)

Y_nu_tribimaximal = np.array([
    [c12_tb, s12_tb, 0],
    [-s12_tb/np.sqrt(2), c12_tb/np.sqrt(2), 1/np.sqrt(2)],
    [s12_tb/np.sqrt(2), -c12_tb/np.sqrt(2), 1/np.sqrt(2)]
]) * 1e-11  # Overall scale for realistic masses

# Diagonal charged lepton Yukawa (approximately diagonal)
Y_ell = np.diag([0.00293, 0.607, 10.2])

# Create predictor and compute
predictor_dirac = PMNSPredictor(rho, majorana=False)
result_dirac = predictor_dirac.predict(Y_nu_tribimaximal, Y_ell)

print(predictor_dirac.summary())

In [ ]:
# Compare to experiment
comparison_dirac = predictor_dirac.compare_to_experiment('NO')

print("\nComparison with NuFIT (Normal Ordering):")
print("-" * 50)
print(f"{'Parameter':<12} {'Predicted':>12} {'Observed':>12} {'Pull':>10}")
print("-" * 50)
for param in ['theta12', 'theta23', 'theta13', 'delta_CP']:
    c = comparison_dirac[param]
    print(f"{param:<12} {c['predicted']:>12.2f} {c['observed']:>12.2f} {c['pull']:>+10.2f}σ")
print("-" * 50)
print(f"Total χ² = {comparison_dirac['chi2']:.2f}")

## 5. Type-I Seesaw Example

In the Type-I seesaw mechanism:

$$M_\nu = -M_D M_R^{-1} M_D^T$$

where $M_D = Y_D \cdot \mathrm{diag}(\rho_1, \rho_2, \rho_3)$.

In [ ]:
# Seesaw parameters
# Dirac Yukawa (hierarchical, similar to up-quarks)
Y_D = np.array([
    [1e-5, 1e-6, 1e-7],
    [1e-6, 1e-3, 1e-4],
    [1e-7, 1e-4, 0.5]
])

# Heavy Majorana mass matrix (diagonal, hierarchical)
M_R = np.diag([1e10, 1e12, 1e14])  # GeV

# Charged lepton Yukawa (same as before)
Y_ell_seesaw = np.diag([0.00293, 0.607, 10.2])

# Create predictor and compute
predictor_seesaw = PMNSPredictor(rho, majorana=True)
result_seesaw = predictor_seesaw.predict(Y_D, Y_ell_seesaw, M_R)

print(predictor_seesaw.summary())

In [ ]:
# Compare to experiment
comparison_seesaw = predictor_seesaw.compare_to_experiment('NO')

print("\nSeesaw: Comparison with NuFIT (Normal Ordering):")
print("-" * 50)
print(f"{'Parameter':<12} {'Predicted':>12} {'Observed':>12} {'Pull':>10}")
print("-" * 50)
for param in ['theta12', 'theta23', 'theta13', 'delta_CP']:
    c = comparison_seesaw[param]
    print(f"{param:<12} {c['predicted']:>12.2f} {c['observed']:>12.2f} {c['pull']:>+10.2f}σ")
print("-" * 50)
print(f"Total χ² = {comparison_seesaw['chi2']:.2f}")

## 6. Visualization: Predicted vs Observed

In [ ]:
# Compare both scenarios
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

params = ['theta12', 'theta23', 'theta13']
param_labels = ['θ₁₂', 'θ₂₃', 'θ₁₃']

# Left: Angles
ax = axes[0]
x = np.arange(len(params))
width = 0.25

obs_vals = [NUFIT_NO[p] for p in params]
obs_errs = [NUFIT_NO[f'{p}_err'] for p in params]
dirac_vals = [result_dirac[p] for p in params]
seesaw_vals = [result_seesaw[p] for p in params]

ax.bar(x - width, obs_vals, width, yerr=obs_errs, label='NuFIT', 
       color='#2c3e50', capsize=5, alpha=0.8)
ax.bar(x, dirac_vals, width, label='Dirac', 
       color='#3498db', alpha=0.8)
ax.bar(x + width, seesaw_vals, width, label='Seesaw', 
       color='#e74c3c', alpha=0.8)

ax.set_ylabel('Angle (degrees)')
ax.set_title('Mixing Angles: Predictions vs Experiment')
ax.set_xticks(x)
ax.set_xticklabels(param_labels, fontsize=14)
ax.legend()

# Right: Pulls
ax = axes[1]
params_all = ['theta12', 'theta23', 'theta13', 'delta_CP']
param_labels_all = ['θ₁₂', 'θ₂₃', 'θ₁₃', 'δ_CP']
x = np.arange(len(params_all))

dirac_pulls = [comparison_dirac[p]['pull'] for p in params_all]
seesaw_pulls = [comparison_seesaw[p]['pull'] for p in params_all]

ax.bar(x - width/2, dirac_pulls, width, label='Dirac', 
       color='#3498db', alpha=0.8)
ax.bar(x + width/2, seesaw_pulls, width, label='Seesaw', 
       color='#e74c3c', alpha=0.8)

ax.axhline(y=0, color='black', linewidth=2)
ax.axhline(y=-2, color='gray', linestyle='--', alpha=0.5)
ax.axhline(y=2, color='gray', linestyle='--', alpha=0.5)
ax.fill_between([-0.5, 3.5], -2, 2, color='green', alpha=0.1)

ax.set_ylabel('Pull (σ)')
ax.set_title('Pulls from Experimental Values')
ax.set_xticks(x)
ax.set_xticklabels(param_labels_all, fontsize=14)
ax.legend()
ax.set_xlim(-0.5, 3.5)

plt.tight_layout()
plt.show()

## 7. Uncertainty Propagation

We propagate uncertainties from input parameters to PMNS observables using Monte Carlo sampling.

In [ ]:
# Monte Carlo uncertainty propagation
n_samples = 500

# Vary rho and Yukawa parameters
rho_base = np.array([0.0003, 0.06, 0.98])
rho_sigma = np.array([0.0001, 0.01, 0.02])  # Assumed uncertainties

results_mc = []
for _ in range(n_samples):
    # Sample rho
    rho_sample = rho_base + rho_sigma * np.random.randn(3)
    rho_sample = np.clip(rho_sample, 0.0001, 0.9999)
    rho_sample = np.sort(rho_sample)  # Maintain ordering
    
    # Sample Yukawa (small perturbations)
    Y_nu_sample = Y_nu_tribimaximal * (1 + 0.1 * np.random.randn(3, 3))
    
    # Compute PMNS
    pred = PMNSPredictor(rho_sample, majorana=False)
    try:
        res = pred.predict(Y_nu_sample, Y_ell)
        results_mc.append(res)
    except:
        pass

# Extract distributions
theta12_samples = [r['theta12'] for r in results_mc]
theta23_samples = [r['theta23'] for r in results_mc]
theta13_samples = [r['theta13'] for r in results_mc]
deltaCP_samples = [r['delta_CP'] for r in results_mc]

print(f"Monte Carlo results ({len(results_mc)} successful samples):")
print(f"  θ₁₂ = {np.mean(theta12_samples):.2f}° ± {np.std(theta12_samples):.2f}°")
print(f"  θ₂₃ = {np.mean(theta23_samples):.2f}° ± {np.std(theta23_samples):.2f}°")
print(f"  θ₁₃ = {np.mean(theta13_samples):.2f}° ± {np.std(theta13_samples):.2f}°")
print(f"  δ_CP = {np.mean(deltaCP_samples):.1f}° ± {np.std(deltaCP_samples):.1f}°")

In [ ]:
# Visualize distributions
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

data = [theta12_samples, theta23_samples, theta13_samples, deltaCP_samples]
labels = ['θ₁₂ (°)', 'θ₂₃ (°)', 'θ₁₃ (°)', 'δ_CP (°)']
exp_vals = [NUFIT_NO['theta12'], NUFIT_NO['theta23'], 
            NUFIT_NO['theta13'], NUFIT_NO['delta_CP']]
exp_errs = [NUFIT_NO['theta12_err'], NUFIT_NO['theta23_err'],
            NUFIT_NO['theta13_err'], NUFIT_NO['delta_CP_err']]

for ax, d, label, exp_val, exp_err in zip(axes.flat, data, labels, exp_vals, exp_errs):
    ax.hist(d, bins=30, density=True, alpha=0.7, color='steelblue', 
            edgecolor='black', label='TSQVT prediction')
    
    # Experimental band
    ax.axvline(exp_val, color='red', linewidth=2, label='NuFIT central')
    ax.axvspan(exp_val - exp_err, exp_val + exp_err, 
               color='red', alpha=0.2, label='NuFIT 1σ')
    
    ax.set_xlabel(label, fontsize=12)
    ax.set_ylabel('Probability density', fontsize=12)
    ax.legend(fontsize=10)

fig.suptitle('TSQVT Predictions vs Experimental Constraints', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

## Summary

The TSQVT framework provides a natural mechanism for neutrino mixing through:

1. **Vacuum structure:** Three minima $\rho_1 < \rho_2 < \rho_3$ determine the generation structure
2. **Mass matrices:** Both Dirac and Majorana scenarios are supported
3. **PMNS prediction:** Mixing angles and CP phase follow from the Yukawa structure

Key findings:
- Simple tribimaximal-like structures give reasonable first approximations
- Non-zero $\theta_{13}$ requires deviations from exact tribimaximal mixing
- The CP phase depends sensitively on the complex phases in Yukawa matrices
- Seesaw mechanism naturally generates small neutrino masses

In [ ]:
# Final summary
print("="*60)
print("TSQVT NEUTRINO MIXING PREDICTIONS - SUMMARY")
print("="*60)
print("\nVacuum condensation values:")
for i, r in enumerate(rho):
    print(f"  ρ_{i+1} = {r:.4f}")
print("\nBest-fit predictions (Dirac scenario):")
print(f"  θ₁₂ = {result_dirac['theta12']:.2f}° (exp: {NUFIT_NO['theta12']:.2f}°)")
print(f"  θ₂₃ = {result_dirac['theta23']:.2f}° (exp: {NUFIT_NO['theta23']:.2f}°)")
print(f"  θ₁₃ = {result_dirac['theta13']:.2f}° (exp: {NUFIT_NO['theta13']:.2f}°)")
print(f"  δ_CP = {result_dirac['delta_CP']:.1f}° (exp: {NUFIT_NO['delta_CP']:.0f}°)")
print("="*60)